In [1]:
import os
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
config = tf.compat.v1.ConfigProto(allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
tf.compat.v1.Session(config=config) 

from sentence_transformers import SentenceTransformer, util
from tensorflow.keras.layers import Layer,Input,Dense,Lambda
from tensorflow.keras import Model
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import tensorflow_hub as hub
import tensorflow_text
from glob import glob
import setting as set_fnc
import copy as cp

use_enc_large = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")
use_enc_small = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla V100-SXM2-32GB-LS, pci bus id: 0000:0a:00.0, compute capability: 7.0



# Load datasets

In [2]:
mode = 'test'
q_lan = ['ru','ko','fi','ja']
corpus = 'XORQA'

df_question = {}
df_paragraph = {}
df_doc = pd.read_csv(f'../datasets/{corpus}/{mode}/{corpus.lower()}_doc_en-ru.csv')
for lan in q_lan:
    df_question.update({
        f'en-{lan}':pd.read_csv(f'../datasets/{corpus}/{mode}/{corpus.lower()}_question_en-{lan}_mbart.csv').drop_duplicates(subset=['question'])
    })  


# mUSE Example

In [4]:
mUSE_model = hub.load(f'../models/{corpus}/finetuned_USE_XORQA_train_en-ru_ko_ja_top0-0_q-d-distillation_1000MSE_0.1MSEq_1.0MSEd_0.1MSEqd_0.001LR_teacher_best_teacher_batchsize_16_acc_metric_3term')
doc_context_id = df_doc['doc_id'].to_list()    
doc_context_encoded = mUSE_model(df_doc['doc'].to_list()).numpy()
for lan in q_lan:
    question_id = df_question[f'en-{lan}']['doc_id'].to_list()
    questions = mUSE_model(df_question[f'en-{lan}']['question'].to_list()).numpy()
    
    top_1,top_5,top_10,mrr = set_fnc.evaluate(question_id,questions,doc_context_id,doc_context_encoded)

    print(f'USE-{lan}')
    precision = top_1 / len(questions)
    print(f"Traninng Score P@1: {precision:.3f}")
    precision = top_5 / len(questions)
    print(f"Traninng Score P@5: {precision:.3f}")
    precision = top_10 / len(questions)
    print(f"Traninng Score P@10: {precision:.3f}")
    print(f"Mrr score:{mrr:.3f}")

  5%|▌         | 26/474 [00:00<00:01, 257.49it/s]

USE-ru
Traninng Score P@1: 0.582
Traninng Score P@5: 0.797
Traninng Score P@10: 0.854
Mrr score:0.671


  8%|▊         | 31/371 [00:00<00:01, 306.56it/s]

USE-ko
Traninng Score P@1: 0.477
Traninng Score P@5: 0.707
Traninng Score P@10: 0.783
Mrr score:0.573


 14%|█▍        | 55/386 [00:00<00:00, 542.65it/s]

USE-fi
Traninng Score P@1: 0.248
Traninng Score P@5: 0.353
Traninng Score P@10: 0.394
Mrr score:0.295


100%|██████████| 386/386 [00:00<00:00, 609.09it/s]

USE-ja
Traninng Score P@1: 0.495
Traninng Score P@5: 0.720
Traninng Score P@10: 0.788
Mrr score:0.591


# Sentence-Transformer Example

In [ ]:
ST_model = SentenceTransformer('XXXXXXX')
for lan in q_lan[:]:
    set_fnc.sent_bert_encode(ST_model,'ST_model',lan,df_doc,df_question)
    print('*'*50)